amt_credit; bureau_AMT_CREDIT_SUM_divide_AMT_CREDIT_SUM_DEBT_min; AMT_ANNUITY

In [1]:
import os
import numpy as np
import pandas as pd
import gc
import time
from contextlib import contextmanager
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import KFold, StratifiedKFold
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
PATH = '/home/kai/data/kaggle/homecredit/'

In [2]:
train = pd.read_pickle(PATH + 'test/train_ori.pkl')
test = pd.read_pickle(PATH + 'test/test_ori.pkl')
# train = pd.read_pickle(PATH + 'test/train_w_prev_2.pkl')
# test = pd.read_pickle(PATH + 'test/test_w_prev_2.pkl')

print(train.shape,test.shape)
train.shape, test.shape
prev = pd.read_pickle(PATH + 'test/prev2curr_0.pkl')
print(prev.shape)
pos = pd.read_pickle(PATH + 'inter/poscash2curr.pkl')
print(pos.shape)
cred = pd.read_pickle(PATH + 'inter/credit2curr.pkl')
print(cred.shape)
install = pd.read_pickle(PATH + 'inter/install2curr.pkl')
print(install.shape)
bureau = pd.read_pickle(PATH + 'inter/bureau2curr.pkl')
print(bureau.shape)

files = [prev,pos,cred,install,bureau]
# files = [pos,cred,install,bureau]
for tmp in files:
#     cols = [col for col in tmp.columns if 'std' not in col]
#     tmp = tmp[cols]
#     print(tmp.shape)
    train = train.merge(tmp, on='SK_ID_CURR', how='left')
    test = test.merge(tmp, on='SK_ID_CURR', how='left')
    print(train.shape,test.shape)



(307511, 329) (48744, 328)
(356255, 818)
(356255, 116)
(356255, 502)
(356255, 60)
(356255, 554)
(307511, 1146) (48744, 1145)
(307511, 1261) (48744, 1260)
(307511, 1762) (48744, 1761)
(307511, 1821) (48744, 1820)
(307511, 2374) (48744, 2373)


In [3]:
def minus_name(col1, col2): return col1 + '_minus_' + col2
def minus(df, col1, col2): return df[col1] - df[col2]

def ratio_name(col1, col2): return col1 + '_divide_' + col2
def ratio(df, col1, col2): return df[col1] / df[col2]

def multiply_name(col1, col2): return col1 + '_times_' + col2
def multiply(df, col1, col2): return df[col1] * df[col2]

def positive_count(df, gp_col, col):
    group = (df[col] > 0).astype('int8')
    group = pd.concat([df[gp_col], group], axis=1).groupby(gp_col)[[col]].sum().reset_index().rename(index=str, columns={col: 'positivecount_'+'_'.join([gp_col, col])})
    return group.astype({gp_col: df[gp_col].dtype})

def count(df, gp_col, col):
    group = df[[gp_col,col]].groupby(gp_col)[[col]].count().reset_index().rename(index=str, columns={col:'count_'+gp_col})
    return group.astype({gp_col: df[gp_col].dtype})

def numerical(df, gp_col, col, agg_fun):
    _df = df.groupby(gp_col)[[col]].agg(agg_fun)
    
    columns = []
    for x in _df.columns.levels[0]:
        for y in _df.columns.levels[1]:
            columns.append('_'.join([x, y]))
    _df.columns = columns
    return _df.reset_index().astype({gp_col: df[gp_col].dtype})

def one_hot(df, gp_col, col):
    return pd.concat([df[gp_col], pd.get_dummies(df[col], prefix='onehot_' + col)], axis=1).groupby(gp_col).sum().reset_index().astype({gp_col: df[gp_col].dtype})

'done'

'done'

In [4]:
df = pd.concat([train, test], axis = 0)  
print(df.shape)

/home/kai/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  """Entry point for launching an IPython kernel.


(356255, 2374)


In [5]:
bureau_amt_col1 = [col for col in df.columns if 'AMT' in col and 'bureau' in col and 'mean' in col and 'PROLONG' not in col]
bureau_amt_col2 = [col for col in df.columns if 'AMT' in col and 'bureau' in col and 'min' not in col and 'PROLONG' not in col and 'divide' not in col and 'sum' not in col and 'std' not in col]

bureau_amt = bureau_amt_col1 +  bureau_amt_col2

In [6]:
# print(len(bureau_amt_col1))
# print(bureau_amt_col1)

In [7]:
train_amt = ['AMT_CREDIT','AMT_ANNUITY']

In [8]:
tmp = 0
print(len(bureau_amt_col1))
for i in bureau_amt_col1:
    for j in train_amt:
#         print(ratio_name(j,i))
        tmp += 1
        df[ratio_name(j,i)] = ratio(df,j,i)

51


In [9]:
# for i in bureau_amt:
#     for j in train_amt:
#         print(minus_name(i,j))
#         df[minus_name(i,j)] = minus(df,i,j)

In [10]:
print(df.shape)

(356255, 2476)


In [11]:
train = df[df['TARGET'].notnull()]
test = df[df['TARGET'].isnull()]
test.drop('TARGET', inplace = True, axis = 1)
print(train.shape, test.shape)

(307511, 2476) (48744, 2475)


/home/kai/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3694: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [12]:
train.to_pickle(PATH +'test/train_bureau1.pkl')
test.to_pickle(PATH +'test/test_bureau1.pkl')

In [ ]:
warnings.simplefilter(action='ignore', category=FutureWarning)
PATH = '/home/kai/data/kaggle/homecredit/'
print('begin cv')
target = train['TARGET']
test_df = test.copy()
ignore_cols = ['ORGANIZATION_TYPE', 'TARGET', 'SK_ID_CURR']
features = [x for x in train.columns if x not in ignore_cols]
train = train[features]
test = test[features]

from sklearn.model_selection import StratifiedKFold
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score
import lightgbm as lgbm

lgbm_train = lgbm.Dataset(data=train,
                          label=target,

                          categorical_feature=[],#categorical_feats,
                          free_raw_data=False)

# lgbm_params = {'task': 'train', 'boosting_type': 'gbdt', 'objective': 'binary', 'metric': 'auc', 
#           'learning_rate': 0.01, 'num_leaves': 48, 'num_iteration': 4000, 'verbose': 0 ,
#           'colsample_bytree':.8, 'subsample':.9, 'max_depth':7, 'reg_alpha':.1, 'reg_lambda':.1, 
#           'min_split_gain':.01, 'min_child_weight':1,'num_threads': 12,'feature_fraction': 0.2,
#               'scale_pos_weight':1}
lgbm_params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'auc',
    'learning_rate': 0.05,
    'num_iteration': 4000,
    'num_threads': 8,

    'num_leaves': int(round(44.368535336628419)),
    'feature_fraction': 0.28231763168020257,
    'bagging_fraction': 0.94901525271474951,
    'max_depth': int(round(8.0430115561596267)),
    'lambda_l1': 0.30680079516647751,
    'lambda_l2': 0.079128660903201031,
    'min_split_gain': 0.054005067457890979,
    'min_child_weight': 98.172643147364937}


cv_results = lgbm.cv(train_set=lgbm_train,
                     params=lgbm_params,
                     nfold=5,
                     early_stopping_rounds=150,
                     verbose_eval=100,
                     metrics=['auc'])

optimum_boost_rounds = np.argmax(cv_results['auc-mean'])
print('Optimum boost rounds = {}'.format(optimum_boost_rounds))
print('Best CV result = {}'.format(np.max(cv_results['auc-mean'])))

clf = lgbm.train(train_set=lgbm_train,
                 params=lgbm_params,
                 num_boost_round=optimum_boost_rounds,
                verbose_eval=100)

""" Predict on test set and create submission """
y_pred = clf.predict(test)
out_df = pd.DataFrame({'SK_ID_CURR': test_df['SK_ID_CURR'], 'TARGET': y_pred})
#     small_len = out_df[out_df['TARGET']< threshold1]
#     large_len = out_df[out_df['TARGET']> threshold2]
#     print('length of small and large,',len(small_len),len(large_len))
print((out_df['TARGET']>0.5).sum(),'more than half')